## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [1]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath)

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

c:\Users\Nihal\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Toplam 1000 film yüklendi.


## Veriyi parçalarına ayırma(Chunking işlemi)


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Toplam {len(docs)} film parçası oluşturuldu.")

Toplam 1000 film parçası oluşturuldu.


## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [4]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

Anahtar başarıyla yüklendi. İlk 4 karakter: AIza


In [5]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909, -0.022324152, -0.05460342, -0.0062845494, -0.033924025]

## ChromaDB üzerine kayıt işlemi

In [6]:
from langchain_chroma import Chroma

In [ ]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [36]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3 })

In [37]:
retrieved_docs = retriever.invoke("Movies starring Meryl Streep")     

len(retrieved_docs)

print(retrieved_docs[1].page_content)


: 513
Movie_Name: Fantastic Mr. Fox
Movie_Rating: 7.9
Movie_Certificate: U
Movie_Year: 2009
Movie_Runtime: 87 min
Movie_Cast: George Clooney, Meryl Streep, Bill Murray, Jason Schwartzman
Movie_Genre: Animation, Adventure, Comedy
Movie_Poster: https://m.media-amazon.com/images/M/MV5BOGUwYTU4NGEtNDM4MS00NDRjLTkwNmQtOTkwMWMyMjhmMjdlXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg
Movie_Description: An urbane fox cannot resist returning to his farm raiding ways and then must help his community survive the farmers' retaliation.
Movie_Poster_HD: https://m.media-amazon.com/images/M/MV5BOGUwYTU4NGEtNDM4MS00NDRjLTkwNmQtOTkwMWMyMjhmMjdlXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_.jpg
All_Movie_Info: fantastic mr fox george clooney meryl streep bill murray jason schwartzman animation adventure comedy an urbane fox cannot resist returning to his farm raiding ways and then must help his community survive the farmers' retaliation


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)


# Intent Classification (Niyet Sınıflandırma) Katmanı


In [44]:
from langchain_core.output_parsers import StrOutputParser

In [45]:
intent_system_prompt = (
    "You are a text classifier. Analyze the user's input and classify it into exactly one of these categories:\n"
    "1. MOVIE_QUERY: If the user is asking for movie recommendations, details about a specific movie, plot, cast, or ratings.\n"
    "2. CHITCHAT: If the user is greeting (hi, hello) or asking general questions like 'how are you'.\n"
    "3. OTHER: Anything else unrelated to movies or greetings.\n\n"
    "Return ONLY the category name (MOVIE_QUERY, CHITCHAT, or OTHER). Do not write anything else."
)

intent_prompt = ChatPromptTemplate.from_messages([
    ("system", intent_system_prompt),
    ("user", "{input}")
])


In [46]:
intent_chain = intent_prompt | llm | StrOutputParser()

In [47]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


In [48]:
system_prompt = (
    "You are a helpful movie recommendation assistant. Use the following movie database context to answer the user's question."
    "Only recommend movies from the provided context"
    "\n"
    "\n"
    "If you don't find suitable movies in the context, say so honestly."
    "\n\n"
    "{context}"
)



In [49]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [50]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri Oluşturma (RAG + LLM)


In [51]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [56]:
response = rag_chain.invoke({"input": "How is the weather?"})
intent = intent_chain.invoke({"input": "How is the weather?"}).strip()
print(response["answer"])
print(f"Detected Intent: {intent}")

I'm sorry, but I cannot provide information about the weather. My purpose is to recommend movies based on the context provided.
Detected Intent: OTHER
